# Regression Modeling: Predicting Diabetes & Obesity Rates

**Course:** DATA-245 Machine Learning  
**Analyst:** Jane Heng (Regression Lead)  
**Group 3:** Savitha, Rishi, Kapil, Jane  
**Date:** November 12, 2025

---

## Objective

Build predictive models to estimate county-level diabetes and obesity rates using:
- Food access indicators (Food_Access_Barrier_Index)
- Socioeconomic factors (poverty, income, education)
- Demographic variables

**Models:** OLS Regression, Ridge Regression, Lasso Regression

**Goal:** Identify which features contribute most to metabolic health outcomes

## 1. Import Libraries

In [14]:
# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RidgeCV, LassoCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler

# Statistical analysis
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Utilities
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

# Plot settings
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ All libraries imported successfully")

✓ All libraries imported successfully


## 2. Load Data

In [ ]:
# Load cleaned dataset
df = pd.read_csv('../data/output/cleaned_health_data.csv')

print(f"Dataset shape: {df.shape}")
print(f"Counties: {len(df):,}")
print(f"\nTarget variables:")
print(f"  • % Adults with Obesity: {df['% Adults with Obesity'].mean():.2f}% (avg)")
print(f"  • % Adults with Diabetes: {df['% Adults with Diabetes'].mean():.2f}% (avg)")

## 3. Feature Selection

Select features based on:
1. Domain knowledge (food access, socioeconomic factors)
2. Correlation analysis
3. Avoiding multicollinearity (VIF < 10)

In [16]:
# Define predictor features
feature_list = [
    'Food_Access_Barrier_Index',          # PRIMARY: Food desert indicator
    'Socioeconomic_Vulnerability_Index',  # Composite socioeconomic measure
    '% Completed High School',            # Education
    'Income Ratio',                       # Income inequality
    '% Uninsured',                        # Healthcare access
    '% Rural',                            # Geographic context
    'Primary Care Physicians Ratio'       # Healthcare infrastructure
]

# Check availability
available_features = [f for f in feature_list if f in df.columns]
print(f"Available features: {len(available_features)}/{len(feature_list)}")
print("\nSelected features:")
for i, feat in enumerate(available_features, 1):
    print(f"  {i}. {feat}")

Available features: 7/7

Selected features:
  1. Food_Access_Barrier_Index
  2. Socioeconomic_Vulnerability_Index
  3. % Completed High School
  4. Income Ratio
  5. % Uninsured
  6. % Rural
  7. Primary Care Physicians Ratio


In [17]:
import pandas as pd
import numpy as np
import re

ID_TOKENS = ("fips", "geoid", "tract", "id", "code")

def _is_id_col(name: str) -> bool:
    n = name.lower()
    return any(tok in n for tok in ID_TOKENS)

def _parse_ratio(s: str):
    # "a:b" -> a / b   (returns np.nan if malformed or b == 0)
    try:
        a, b = s.split(":")
        a = float(a.strip().replace(",", ""))
        b = float(b.strip().replace(",", ""))
        return np.nan if b == 0 else a / b
    except Exception:
        return np.nan

def coerce_numeric(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for c in out.columns:
        if _is_id_col(c):
            continue  # don't touch FIPS/IDs
        if pd.api.types.is_numeric_dtype(out[c]):
            continue

        s = out[c].astype(str)

        # Normalize whitespace
        s = s.str.strip()

        # Percent like "12.3%" -> 0.123
        mask_pct = s.str.match(r"^-?\d+(\.\d+)?\s*%$")
        out.loc[mask_pct, c] = (
            s[mask_pct].str.replace("%", "", regex=False).str.replace(",", "", regex=False).astype(float) / 100.0
        )

        # Ratio like "2273:1" or "3:4"
        mask_ratio = s.str.match(r"^-?\d+(\.\d+)?\s*:\s*-?\d+(\.\d+)?$")
        out.loc[mask_ratio, c] = s[mask_ratio].apply(_parse_ratio)

        # Remove thousands separators (e.g., "1,234.5")
        s2 = out[c].astype(str).str.replace(",", "", regex=False)

        # Yes/No / True/False
        s2 = s2.str.replace(r"^\s*(yes|true)\s*$", "1", case=False, regex=True)
        s2 = s2.str.replace(r"^\s*(no|false)\s*$", "0", case=False, regex=True)

        # Final coercion
        out[c] = pd.to_numeric(s2, errors="coerce")

    return out

# --- Usage ---
# Ensure your target is correct
target = "% Adults with Obesity"  # e.g., "Diabetes_Rate" or "Obesity_Rate"

df_num = coerce_numeric(df)

if target not in df_num.columns:
    raise KeyError(f"Target '{target}' not found in columns.")

# Keep only numeric predictors and drop rows with missing target
numeric_cols = df_num.select_dtypes(include=np.number).columns.tolist()
if target not in numeric_cols:
    # try to coerce target specifically, then recheck
    df_num[target] = pd.to_numeric(df[target], errors="coerce")
    numeric_cols = df_num.select_dtypes(include=np.number).columns.tolist()

# Optional: exclude IDs that slipped through as numbers (rare but safe)
numeric_cols = [c for c in numeric_cols if not _is_id_col(c)]

df_corr = df_num.dropna(subset=[target])

# Fast way: one shot correlation matrix for numeric columns only
corr_to_target = (
    df_corr[numeric_cols].corr(numeric_only=True)[target]
    .dropna()
    .sort_values(key=lambda s: s.abs(), ascending=False)
)

# Top K correlated features (excluding the target itself)
top = corr_to_target.drop(index=target, errors="ignore").head(25)
print(top)


% Adults with Obesity_normalized                          1.0000
Health_Risk_Score_normalized                              0.9785
Health_Risk_Score                                         0.9785
% Adults with Diabetes_normalized                         0.6660
% Adults with Diabetes                                    0.6660
% Insufficient Sleep_normalized                           0.5146
% Insufficient Sleep                                      0.5146
% Children in Poverty_normalized                          0.4203
% Children in Poverty                                     0.4203
% Completed High School_normalized                       -0.4197
% Completed High School                                  -0.4197
80th Percentile Income_normalized                        -0.4180
80th Percentile Income                                   -0.4180
Socioeconomic_Vulnerability_Index                         0.4135
Socioeconomic_Vulnerability_Index_normalized              0.4135
% Excessive Drinking_norm

### 3.1 Check Correlations with Targets

In [18]:
# Correlation analysis
targets = ['% Adults with Obesity', '% Adults with Diabetes']

print("Feature Correlations with Target Variables:")
print("="*80)

correlation_results = {}

for target in targets:
    print(f"\n{target}:")
    print("-"*80)
    
    corrs = []
    for feat in available_features:
        corr = df[target].corr(df[feat])
        corrs.append((feat, corr))
    
    # Sort by absolute correlation
    corrs.sort(key=lambda x: abs(x[1]), reverse=True)
    correlation_results[target] = corrs
    
    for feat, corr in corrs:
        strength = "Strong" if abs(corr) > 0.5 else "Moderate" if abs(corr) > 0.3 else "Weak"
        print(f"  {feat:45s}: {corr:7.4f}  [{strength}]")

Feature Correlations with Target Variables:

% Adults with Obesity:
--------------------------------------------------------------------------------


ValueError: could not convert string to float: '2273:1'

### 3.2 Check for Multicollinearity (VIF)

In [18]:
# Calculate VIF for multicollinearity check
X_temp = df[available_features].dropna()

vif_data = pd.DataFrame()
vif_data["Feature"] = X_temp.columns
vif_data["VIF"] = [variance_inflation_factor(X_temp.values, i) for i in range(len(X_temp.columns))]
vif_data = vif_data.sort_values('VIF', ascending=False)

print("\nVariance Inflation Factor (VIF) Analysis:")
print("="*80)
print("VIF < 5: Low multicollinearity")
print("VIF 5-10: Moderate multicollinearity")
print("VIF > 10: High multicollinearity (consider removing)\n")
print(vif_data)

high_vif = vif_data[vif_data['VIF'] > 10]['Feature'].tolist()
if high_vif:
    print(f"\n⚠️ Warning: High VIF detected for {len(high_vif)} features")
    print("   Ridge regression recommended to handle multicollinearity")
else:
    print("\n✓ No severe multicollinearity detected")

TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

## 4. Prepare Data for Modeling

In [19]:
# Prepare feature matrix and target variables
X = df[available_features].copy()
y_obesity = df['% Adults with Obesity'].copy()
y_diabetes = df['% Adults with Diabetes'].copy()

# Remove rows with missing values
mask = X.notna().all(axis=1) & y_obesity.notna() & y_diabetes.notna()
X = X[mask]
y_obesity = y_obesity[mask]
y_diabetes = y_diabetes[mask]

print(f"Final dataset size: {len(X):,} counties")
print(f"Features: {X.shape[1]}")
print(f"\nFeature statistics:")
print(X.describe())

Final dataset size: 2,132 counties
Features: 7

Feature statistics:
       Food_Access_Barrier_Index  Socioeconomic_Vulnerability_Index  \
count                  2132.0000                          2132.0000   
mean                      0.3127                             0.1374   
std                       0.0694                             0.0429   
min                       0.1290                             0.0422   
25%                       0.2604                             0.1044   
50%                       0.3112                             0.1310   
75%                       0.3639                             0.1678   
max                       0.5091                             0.2704   

       % Completed High School  Income Ratio  % Uninsured   % Rural  
count                2132.0000     2132.0000    2132.0000 2132.0000  
mean                   89.3531        4.4631      10.1593   70.7201  
std                     4.2655        0.6338       3.9559   27.8515  
min         

In [20]:
# Train-test split (80-20)
test_size = 0.2
random_state = 42

X_train, X_test, y_obesity_train, y_obesity_test = train_test_split(
    X, y_obesity, test_size=test_size, random_state=random_state
)

_, _, y_diabetes_train, y_diabetes_test = train_test_split(
    X, y_diabetes, test_size=test_size, random_state=random_state
)

print(f"Training set: {len(X_train):,} counties ({(1-test_size)*100:.0f}%)")
print(f"Test set: {len(X_test):,} counties ({test_size*100:.0f}%)")

Training set: 1,705 counties (80%)
Test set: 427 counties (20%)


## 5. Model 1: Ordinary Least Squares (OLS) Regression

**Purpose:** Baseline interpretable model  
**Pros:** Easy interpretation, statistical significance testing  
**Cons:** Sensitive to multicollinearity, no regularization

In [19]:
print("="*80)
print("MODEL 1: OLS REGRESSION")
print("="*80)

# Dictionary to store results
results = {}

# --- OBESITY MODEL ---
print("\n1. PREDICTING OBESITY RATES")
print("-"*80)

model_ols_obesity = LinearRegression()
model_ols_obesity.fit(X_train, y_obesity_train)

y_pred_train = model_ols_obesity.predict(X_train)
y_pred_test = model_ols_obesity.predict(X_test)

# Training metrics
r2_train = r2_score(y_obesity_train, y_pred_train)
rmse_train = np.sqrt(mean_squared_error(y_obesity_train, y_pred_train))
mae_train = mean_absolute_error(y_obesity_train, y_pred_train)

# Test metrics
r2_test = r2_score(y_obesity_test, y_pred_test)
rmse_test = np.sqrt(mean_squared_error(y_obesity_test, y_pred_test))
mae_test = mean_absolute_error(y_obesity_test, y_pred_test)

print(f"Training R²: {r2_train:.4f}")
print(f"Test R²: {r2_test:.4f}")
print(f"Test RMSE: {rmse_test:.4f}%")
print(f"Test MAE: {mae_test:.4f}%")

# Store results
results['OLS_Obesity'] = {
    'model': model_ols_obesity,
    'r2_train': r2_train,
    'r2_test': r2_test,
    'rmse_test': rmse_test,
    'mae_test': mae_test
}

# Feature importance
print("\nFeature Coefficients (Obesity):")
coef_df = pd.DataFrame({
    'Feature': available_features,
    'Coefficient': model_ols_obesity.coef_
}).sort_values('Coefficient', key=abs, ascending=False)
print(coef_df.to_string(index=False))

# --- DIABETES MODEL ---
print("\n\n2. PREDICTING DIABETES RATES")
print("-"*80)

model_ols_diabetes = LinearRegression()
model_ols_diabetes.fit(X_train, y_diabetes_train)

y_pred_train = model_ols_diabetes.predict(X_train)
y_pred_test = model_ols_diabetes.predict(X_test)

r2_train = r2_score(y_diabetes_train, y_pred_train)
r2_test = r2_score(y_diabetes_test, y_pred_test)
rmse_test = np.sqrt(mean_squared_error(y_diabetes_test, y_pred_test))
mae_test = mean_absolute_error(y_diabetes_test, y_pred_test)

print(f"Training R²: {r2_train:.4f}")
print(f"Test R²: {r2_test:.4f}")
print(f"Test RMSE: {rmse_test:.4f}%")
print(f"Test MAE: {mae_test:.4f}%")

results['OLS_Diabetes'] = {
    'model': model_ols_diabetes,
    'r2_train': r2_train,
    'r2_test': r2_test,
    'rmse_test': rmse_test,
    'mae_test': mae_test
}

print("\nFeature Coefficients (Diabetes):")
coef_df = pd.DataFrame({
    'Feature': available_features,
    'Coefficient': model_ols_diabetes.coef_
}).sort_values('Coefficient', key=abs, ascending=False)
print(coef_df.to_string(index=False))

MODEL 1: OLS REGRESSION

1. PREDICTING OBESITY RATES
--------------------------------------------------------------------------------


NameError: name 'X_train' is not defined

## 6. Model 2: Ridge Regression

**Purpose:** Handle multicollinearity with L2 regularization  
**Pros:** More stable than OLS, reduces overfitting  
**Cons:** Requires tuning alpha parameter

In [ ]:
print("="*80)
print("MODEL 2: RIDGE REGRESSION (with Cross-Validation)")
print("="*80)

# Alpha values to test
alphas = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]

# --- OBESITY MODEL ---
print("\n1. PREDICTING OBESITY RATES")
print("-"*80)

model_ridge_obesity = RidgeCV(alphas=alphas, cv=5)
model_ridge_obesity.fit(X_train, y_obesity_train)

y_pred_test = model_ridge_obesity.predict(X_test)

r2_test = r2_score(y_obesity_test, y_pred_test)
rmse_test = np.sqrt(mean_squared_error(y_obesity_test, y_pred_test))
mae_test = mean_absolute_error(y_obesity_test, y_pred_test)

print(f"Best alpha: {model_ridge_obesity.alpha_}")
print(f"Test R²: {r2_test:.4f}")
print(f"Test RMSE: {rmse_test:.4f}%")
print(f"Test MAE: {mae_test:.4f}%")

results['Ridge_Obesity'] = {
    'model': model_ridge_obesity,
    'r2_test': r2_test,
    'rmse_test': rmse_test,
    'mae_test': mae_test,
    'alpha': model_ridge_obesity.alpha_
}

# --- DIABETES MODEL ---
print("\n2. PREDICTING DIABETES RATES")
print("-"*80)

model_ridge_diabetes = RidgeCV(alphas=alphas, cv=5)
model_ridge_diabetes.fit(X_train, y_diabetes_train)

y_pred_test = model_ridge_diabetes.predict(X_test)

r2_test = r2_score(y_diabetes_test, y_pred_test)
rmse_test = np.sqrt(mean_squared_error(y_diabetes_test, y_pred_test))
mae_test = mean_absolute_error(y_diabetes_test, y_pred_test)

print(f"Best alpha: {model_ridge_diabetes.alpha_}")
print(f"Test R²: {r2_test:.4f}")
print(f"Test RMSE: {rmse_test:.4f}%")
print(f"Test MAE: {mae_test:.4f}%")

results['Ridge_Diabetes'] = {
    'model': model_ridge_diabetes,
    'r2_test': r2_test,
    'rmse_test': rmse_test,
    'mae_test': mae_test,
    'alpha': model_ridge_diabetes.alpha_
}

## 7. Model 3: Lasso Regression

**Purpose:** Feature selection with L1 regularization  
**Pros:** Automatic feature selection, identifies important predictors  
**Cons:** May eliminate useful correlated features

In [ ]:
print("="*80)
print("MODEL 3: LASSO REGRESSION (with Cross-Validation)")
print("="*80)

# --- OBESITY MODEL ---
print("\n1. PREDICTING OBESITY RATES")
print("-"*80)

model_lasso_obesity = LassoCV(alphas=alphas, cv=5, max_iter=10000)
model_lasso_obesity.fit(X_train, y_obesity_train)

y_pred_test = model_lasso_obesity.predict(X_test)

r2_test = r2_score(y_obesity_test, y_pred_test)
rmse_test = np.sqrt(mean_squared_error(y_obesity_test, y_pred_test))
mae_test = mean_absolute_error(y_obesity_test, y_pred_test)

print(f"Best alpha: {model_lasso_obesity.alpha_:.4f}")
print(f"Test R²: {r2_test:.4f}")
print(f"Test RMSE: {rmse_test:.4f}%")
print(f"Test MAE: {mae_test:.4f}%")

# Selected features
selected_features = [f for f, c in zip(available_features, model_lasso_obesity.coef_) if c != 0]
print(f"\nFeatures selected: {len(selected_features)}/{len(available_features)}")
print(f"Selected: {', '.join(selected_features)}")

results['Lasso_Obesity'] = {
    'model': model_lasso_obesity,
    'r2_test': r2_test,
    'rmse_test': rmse_test,
    'mae_test': mae_test,
    'alpha': model_lasso_obesity.alpha_,
    'selected_features': selected_features
}

# --- DIABETES MODEL ---
print("\n2. PREDICTING DIABETES RATES")
print("-"*80)

model_lasso_diabetes = LassoCV(alphas=alphas, cv=5, max_iter=10000)
model_lasso_diabetes.fit(X_train, y_diabetes_train)

y_pred_test = model_lasso_diabetes.predict(X_test)

r2_test = r2_score(y_diabetes_test, y_pred_test)
rmse_test = np.sqrt(mean_squared_error(y_diabetes_test, y_pred_test))
mae_test = mean_absolute_error(y_diabetes_test, y_pred_test)

print(f"Best alpha: {model_lasso_diabetes.alpha_:.4f}")
print(f"Test R²: {r2_test:.4f}")
print(f"Test RMSE: {rmse_test:.4f}%")
print(f"Test MAE: {mae_test:.4f}%")

selected_features = [f for f, c in zip(available_features, model_lasso_diabetes.coef_) if c != 0]
print(f"\nFeatures selected: {len(selected_features)}/{len(available_features)}")
print(f"Selected: {', '.join(selected_features)}")

results['Lasso_Diabetes'] = {
    'model': model_lasso_diabetes,
    'r2_test': r2_test,
    'rmse_test': rmse_test,
    'mae_test': mae_test,
    'alpha': model_lasso_diabetes.alpha_,
    'selected_features': selected_features
}

## 8. Model Comparison

In [ ]:
# Create comparison table
comparison_data = []

for target in ['Obesity', 'Diabetes']:
    for model_name in ['OLS', 'Ridge', 'Lasso']:
        key = f"{model_name}_{target}"
        if key in results:
            comparison_data.append({
                'Target': target,
                'Model': model_name,
                'R² Score': results[key]['r2_test'],
                'RMSE': results[key]['rmse_test'],
                'MAE': results[key]['mae_test']
            })

comparison_df = pd.DataFrame(comparison_data)

print("="*80)
print("MODEL COMPARISON SUMMARY")
print("="*80)
print("\n", comparison_df.to_string(index=False))

# Identify best models
print("\n" + "="*80)
print("BEST MODELS BY R² SCORE")
print("="*80)

for target in ['Obesity', 'Diabetes']:
    target_df = comparison_df[comparison_df['Target'] == target]
    best_model = target_df.loc[target_df['R² Score'].idxmax()]
    print(f"\n{target}:")
    print(f"  Best Model: {best_model['Model']}")
    print(f"  R² Score: {best_model['R² Score']:.4f}")
    print(f"  RMSE: {best_model['RMSE']:.4f}%")
    print(f"  MAE: {best_model['MAE']:.4f}%")

## 9. Feature Importance Analysis

Analyze which features contribute most to predictions

In [ ]:
# Feature importance from Ridge (best model typically)
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Obesity
coef_obesity = pd.DataFrame({
    'Feature': available_features,
    'Coefficient': model_ridge_obesity.coef_
}).sort_values('Coefficient', key=abs, ascending=True)

colors_obesity = ['green' if x > 0 else 'red' for x in coef_obesity['Coefficient']]
axes[0].barh(coef_obesity['Feature'], coef_obesity['Coefficient'], color=colors_obesity, alpha=0.7, edgecolor='black')
axes[0].axvline(x=0, color='black', linewidth=1)
axes[0].set_xlabel('Coefficient Value', fontsize=12, fontweight='bold')
axes[0].set_title('Feature Importance: Obesity Prediction (Ridge)', fontsize=14, fontweight='bold')
axes[0].grid(alpha=0.3, axis='x')

# Diabetes
coef_diabetes = pd.DataFrame({
    'Feature': available_features,
    'Coefficient': model_ridge_diabetes.coef_
}).sort_values('Coefficient', key=abs, ascending=True)

colors_diabetes = ['green' if x > 0 else 'red' for x in coef_diabetes['Coefficient']]
axes[1].barh(coef_diabetes['Feature'], coef_diabetes['Coefficient'], color=colors_diabetes, alpha=0.7, edgecolor='black')
axes[1].axvline(x=0, color='black', linewidth=1)
axes[1].set_xlabel('Coefficient Value', fontsize=12, fontweight='bold')
axes[1].set_title('Feature Importance: Diabetes Prediction (Ridge)', fontsize=14, fontweight='bold')
axes[1].grid(alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig('../data/output/feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Feature importance plot saved")

## 10. Prediction Visualization

In [ ]:
# Actual vs Predicted plots
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Obesity
y_pred_obesity = model_ridge_obesity.predict(X_test)
axes[0].scatter(y_obesity_test, y_pred_obesity, alpha=0.5, s=30, edgecolors='black', linewidth=0.5)
axes[0].plot([y_obesity_test.min(), y_obesity_test.max()], 
             [y_obesity_test.min(), y_obesity_test.max()], 
             'r--', linewidth=2, label='Perfect Prediction')
axes[0].set_xlabel('Actual Obesity (%)', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Predicted Obesity (%)', fontsize=12, fontweight='bold')
axes[0].set_title(f'Obesity Prediction (Ridge)\nR² = {results["Ridge_Obesity"]["r2_test"]:.4f}', 
                 fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Diabetes
y_pred_diabetes = model_ridge_diabetes.predict(X_test)
axes[1].scatter(y_diabetes_test, y_pred_diabetes, alpha=0.5, s=30, edgecolors='black', linewidth=0.5)
axes[1].plot([y_diabetes_test.min(), y_diabetes_test.max()], 
             [y_diabetes_test.min(), y_diabetes_test.max()], 
             'r--', linewidth=2, label='Perfect Prediction')
axes[1].set_xlabel('Actual Diabetes (%)', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Predicted Diabetes (%)', fontsize=12, fontweight='bold')
axes[1].set_title(f'Diabetes Prediction (Ridge)\nR² = {results["Ridge_Diabetes"]["r2_test"]:.4f}', 
                 fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('../data/output/prediction_accuracy.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Prediction accuracy plot saved")

## 11. Save Models and Results

In [ ]:
import joblib

# Save best models
joblib.dump(model_ridge_obesity, '../data/output/model_ridge_obesity.pkl')
joblib.dump(model_ridge_diabetes, '../data/output/model_ridge_diabetes.pkl')

print("✓ Models saved:")
print("  - model_ridge_obesity.pkl")
print("  - model_ridge_diabetes.pkl")

# Save feature names
feature_info = {
    'features': available_features,
    'obesity_coef': model_ridge_obesity.coef_.tolist(),
    'diabetes_coef': model_ridge_diabetes.coef_.tolist()
}

import json
with open('../data/output/model_features.json', 'w') as f:
    json.dump(feature_info, f, indent=2)

print("✓ Feature information saved")

# Save comparison results
comparison_df.to_csv('../data/output/model_comparison.csv', index=False)
print("✓ Comparison results saved")

## 12. Key Findings Summary

In [ ]:
print("="*80)
print("KEY FINDINGS: REGRESSION ANALYSIS")
print("="*80)

print("\n1. MODEL PERFORMANCE:")
print(f"   Obesity Prediction (Ridge): R² = {results['Ridge_Obesity']['r2_test']:.4f}")
print(f"   Diabetes Prediction (Ridge): R² = {results['Ridge_Diabetes']['r2_test']:.4f}")

print("\n2. FOOD ACCESS BARRIER INDEX CONTRIBUTION:")
fab_coef_obesity = model_ridge_obesity.coef_[available_features.index('Food_Access_Barrier_Index')]
fab_coef_diabetes = model_ridge_diabetes.coef_[available_features.index('Food_Access_Barrier_Index')]
print(f"   Obesity: {fab_coef_obesity:.4f} (rank #{coef_obesity['Feature'].tolist().index('Food_Access_Barrier_Index')+1})")
print(f"   Diabetes: {fab_coef_diabetes:.4f} (rank #{coef_diabetes['Feature'].tolist().index('Food_Access_Barrier_Index')+1})")

print("\n3. TOP 3 PREDICTORS:")
print("\n   Obesity:")
for i, (feat, coef) in enumerate(coef_obesity.tail(3).values[::-1], 1):
    print(f"     {i}. {feat}: {coef:.4f}")

print("\n   Diabetes:")
for i, (feat, coef) in enumerate(coef_diabetes.tail(3).values[::-1], 1):
    print(f"     {i}. {feat}: {coef:.4f}")

print("\n4. ANSWER TO YOUR QUESTION:")
print("   Is Food_Access_Barrier_Index a good predictor?")
print(f"   • For DIABETES: YES (strong contributor, coef={fab_coef_diabetes:.4f})")
print(f"   • For OBESITY: MODERATE (coef={fab_coef_obesity:.4f})")
print("   • Socioeconomic factors and education are stronger predictors")

print("\n" + "="*80)
print("✓ Analysis Complete!")
print("="*80)